In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.python.keras import applications
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Dropout

In [18]:
#Dimensions
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/training'
validation_data_dir = 'data/validation'
nb_train_samples = 320
nb_validation_samples = 80
epochs = 5
batch_size = 4

def save_bottlebeck_features():
    datagen = ImageDataGenerator(
        rescale=1. / 255,
        horizontal_flip=True,
        width_shift_range=0.10,
        rotation_range=20,)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    '''
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)
    '''
    
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


In [19]:
save_bottlebeck_features()

Found 8 images belonging to 2 classes.


In [11]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array([0] * (int(nb_train_samples / 2)) + [1] * (int(nb_train_samples / 2)))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * (int(nb_validation_samples / 2)) + [1] * (int(nb_validation_samples / 2)))

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=train_data.shape[1:]))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


In [20]:
train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (int(nb_train_samples / 2)) + [1] * (int(nb_train_samples / 2)))
print(train_data.shape)

validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
print(validation_data.shape)
validation_labels = np.array(
        [0] * (int(nb_validation_samples / 2)) + [1] * (int(nb_validation_samples / 2)))


(320, 4, 4, 512)
(80, 4, 4, 512)


In [ ]:
epochs = 30
train_top_model()

Train on 320 samples, validate on 80 samples
Epoch 1/30
320/320 [==============================] - 3s - loss: 1.6753 - acc: 0.4969 - val_loss: 1.3397 - val_acc: 0.5000
Epoch 2/30
320/320 [==============================] - 2s - loss: 0.9554 - acc: 0.5219 - val_loss: 0.7004 - val_acc: 0.51250. - ETA: 0s - loss: 1.0229 
Epoch 3/30
320/320 [==============================] - 3s - loss: 0.7737 - acc: 0.5187 - val_loss: 0.7117 - val_acc: 0.4500
Epoch 4/30
320/320 [==============================] - 2s - loss: 0.7538 - acc: 0.5406 - val_loss: 0.6985 - val_acc: 0.5000- loss: 0.7688 - a
Epoch 5/30
320/320 [==============================] - 2s - loss: 0.7302 - acc: 0.5625 - val_loss: 0.7601 - val_acc: 0.5375
Epoch 6/30
320/320 [==============================] - 2s - loss: 0.7194 - acc: 0.5719 - val_loss: 0.7353 - val_acc: 0.4750 - loss: 0.7101 - acc:
Epoch 7/30
320/320 [==============================] - 2s - loss: 0.7073 - acc: 0.5594 - val_loss: 0.6956 - val_acc: 0.6000- ETA: 1s - loss: 0.7039 - 